In [27]:
import pandas as pd
import xarray as xr
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import matplotlib as mpl

In [130]:
ICOS_location = "/home/khanalp/data/ICOS2025"

In [131]:
#List the folder names in the ICOS_location directory
folders = [f for f in os.listdir(ICOS_location) if os.path.isdir(os.path.join(ICOS_location, f))]

In [132]:
# write station names as a dictionary with station as key and folder names as values
station_folders = {f.split('_')[0]: f for f in folders}
# write station names as a list
stations = list(station_folders.keys()) 

In [133]:

summary_list = []

for station in stations:
    file_path = os.path.join(ICOS_location, station, f'ICOSETC_{station}_FLUXNET_HH_L2.csv')
    
    if os.path.exists(file_path):
        df_insitu = pd.read_csv(file_path)
        df_insitu['time'] = pd.to_datetime(df_insitu['TIMESTAMP_START'], format='%Y%m%d%H%M')
        df_insitu.set_index('time', inplace=True)
        start_time, end_time = df_insitu.index.min(), df_insitu.index.max()
        
        summary_list.append({
            'station_name': station,
            'start_date': start_time,
            'end_date': end_time
        })
    else:
        print(f"File not found for station: {station}")

# Create summary DataFrame
df_summary = pd.DataFrame(summary_list)

File not found for station: CH-BaK
File not found for station: FI-Kvr
File not found for station: FI-Kmp
File not found for station: IT-OXm
File not found for station: IT-Sas
File not found for station: GR-HeK
File not found for station: DE-BeR
File not found for station: GR-HeM
File not found for station: FI-Tvm


In [134]:
station_details = '/home/khanalp/code/PhD/preprocessICOSdata/csvs/02_station_with_elevation_heightcanopy.csv'

In [135]:
df_station_details_old = pd.read_csv(station_details)

In [136]:
df_station_details_ICOS = pd.read_csv("/home/khanalp/data/stationsICOSdetails.csv")

In [137]:
df_station_details_ICOS['station_name'] = df_station_details_ICOS['Id'].apply(lambda x: x.split('/')[-1].split('_')[-1])


In [138]:
df_station_details_ICOS[['longitude', 'latitude']] = df_station_details_ICOS['Position'].str.split(expand=True).astype(float)


In [139]:
columns_to_keep = ['station_name', 'Site type', 'Elevation above sea', 'longitude', 'latitude']
df_details_subset = df_station_details_ICOS[columns_to_keep]

df_merged = df_summary.merge(df_details_subset, on='station_name', how='left')


In [141]:
df_merged

,station_name,start_date,end_date,Site type,Elevation above sea,longitude,latitude
0,DE-RuS,2019-01-01,2024-12-31 23:30:00,croplands,106.0,6.447145,50.865906
1,IT-Lsn,2016-01-01,2024-12-31 23:30:00,open shrublands,1.0,12.750297,45.740482
2,FR-EM2,2017-01-01,2024-12-31 23:30:00,croplands,85.0,3.020650,49.872110
3,GL-ZaF,2023-01-01,2024-12-31 23:30:00,permanent wetlands,42.0,-20.555773,74.481520
4,FR-CLt,2019-01-01,2024-12-31 23:30:00,grasslands,2050.6,6.410530,45.041400
...,...,...,...,...,...,...,...
65,FR-Pue,2021-01-01,2024-12-31 23:30:00,evergreen broadleaf forests,271.0,3.595700,43.741300
66,IT-Ren,2021-01-01,2024-12-31 23:30:00,evergreen needleleaf forests,1744.0,11.433690,46.586860
67,IT-TrF,2012-01-01,2024-12-31 23:30:00,deciduous needleleaf forests,2100.0,7.560890,45.823760
68,DK-Sor,2021-01-01,2024-12-31 23:30:00,deciduous broadleaf forests,55.0,11.644645,55.485870


In [142]:
df_station_details_old

,station_name,latitude,longitude,elevation,height_canopy_ETH,sd_height_canopy,height_canopy_field_information,measurement_height,IGBP_long_name,IGBP_short_name,Remarks for height_canopy and measurement_height
0,BE-Lon,50.551620,4.746234,168.590302,0,0,1.0,2.70,Croplands,CRO,https://www.sciencedirect.com/science/article/...
1,CH-Oe2,47.286417,7.733750,451.908020,0,0,1.0,2.00,Croplands,CRO,https://www.sciencedirect.com/science/article/...
2,CZ-KrP,49.573257,15.078773,543.737488,0,0,1.0,10.00,Croplands,CRO,https://www.sciencedirect.com/science/article/...
3,DE-Geb,51.099730,10.914630,161.366486,0,0,1.0,2.00,Croplands,CRO,https://www.sciencedirect.com/science/article/...
4,DE-Kli,50.893060,13.522380,480.198456,0,1,1.0,2.00,Croplands,CRO,https://www.sciencedirect.com/science/article/...
...,...,...,...,...,...,...,...,...,...,...,...
66,DE-Akm,53.866170,13.683420,0.241400,13,6,4.0,NaN,Permanent Wetlands,WET,NaN
67,FI-Sii,61.832650,24.192850,166.573929,4,2,0.3,2.00,Permanent Wetlands,WET,https://www.sciencedirect.com/science/article/...
68,FR-LGt,47.322918,2.284102,154.741699,15,7,1.0,2.00,Permanent Wetlands,WET,https://www.sciencedirect.com/science/article/...
69,SE-Deg,64.182030,19.556540,265.453461,3,2,0.3,1.75,Permanent Wetlands,WET,https://www.sciencedirect.com/science/article/...


In [143]:
columns_to_keep = ['station_name','height_canopy_field_information', 'measurement_height', 'IGBP_short_name', 'Remarks for height_canopy and measurement_height']
df_station_details_old_subset = df_station_details_old[columns_to_keep]
df_merged_merged = df_merged.merge(df_station_details_old_subset, on='station_name', how='left')


In [144]:
df_merged_merged

,station_name,start_date,end_date,Site type,Elevation above sea,longitude,latitude,height_canopy_field_information,measurement_height,IGBP_short_name,Remarks for height_canopy and measurement_height
0,DE-RuS,2019-01-01,2024-12-31 23:30:00,croplands,106.0,6.447145,50.865906,1.0,2.0,CRO,https://www.sciencedirect.com/science/article/...
1,IT-Lsn,2016-01-01,2024-12-31 23:30:00,open shrublands,1.0,12.750297,45.740482,2.0,5.0,OSH,https://www.sciencedirect.com/science/article/...
2,FR-EM2,2017-01-01,2024-12-31 23:30:00,croplands,85.0,3.020650,49.872110,NaN,NaN,NaN,NaN
3,GL-ZaF,2023-01-01,2024-12-31 23:30:00,permanent wetlands,42.0,-20.555773,74.481520,NaN,NaN,NaN,NaN
4,FR-CLt,2019-01-01,2024-12-31 23:30:00,grasslands,2050.6,6.410530,45.041400,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
65,FR-Pue,2021-01-01,2024-12-31 23:30:00,evergreen broadleaf forests,271.0,3.595700,43.741300,NaN,NaN,NaN,NaN
66,IT-Ren,2021-01-01,2024-12-31 23:30:00,evergreen needleleaf forests,1744.0,11.433690,46.586860,25.0,35.0,ENF,https://www.sciencedirect.com/science/article/...
67,IT-TrF,2012-01-01,2024-12-31 23:30:00,deciduous needleleaf forests,2100.0,7.560890,45.823760,NaN,NaN,NaN,NaN
68,DK-Sor,2021-01-01,2024-12-31 23:30:00,deciduous broadleaf forests,55.0,11.644645,55.485870,10.0,30.0,DBF,https://www.sciencedirect.com/science/article/...


In [145]:
df_missing_info = df_merged_merged[df_merged_merged.isna().any(axis=1)]


In [146]:
df_missing_info['station_name'].to_list()

['FR-EM2',
 'GL-ZaF',
 'FR-CLt',
 'IT-BFt',
 'UK-AMo',
 'GL-Dsk',
 'SE-Sto',
 'FR-Lqu',
 'IT-Cp2',
 'DE-Har',
 'DK-RCW',
 'DE-Hzd',
 'DE-Amv',
 'FR-Mej',
 'FR-Lus',
 'DE-Brs',
 'CD-Ygb',
 'NO-Hur',
 'ES-LMa',
 'GL-NuF',
 'DE-Msr',
 'IT-Niv',
 'DE-GsB',
 'IT-Noe',
 'GL-ZaH',
 'FI-Sod',
 'FR-Pue',
 'IT-TrF']

In [149]:
df_merged_merged.to_csv('/home/khanalp/code/PhD/preprocessICOSdata/csvs/00_stationinfo_L0_ICOS2025.csv', index=False)

In [4]:
df = pd.read_csv('/home/khanalp/code/PhD/preprocessICOSdata/csvs/01_stationinfo_L0_ICOS2025_with_heights.csv')

In [15]:
f = [f for f in os.listdir('/home/khanalp/data/processed/input_pystemmus/v3') if f.endswith('.nc') and station in f]
    

## For the quality checking of the insitu data

In [31]:
insitu_forcing_path = '/home/khanalp/data/processed/input_pystemmus/v3/'
output_path = "/home/khanalp/code/PhD/preprocessICOSdata/plotting/plots_input_data/v3/forcingquality/"

In [28]:
# Publication-style settings
mpl.rcParams.update({
    "font.size": 11,
    "axes.labelsize": 12,
    "axes.titlesize": 13,
    "legend.fontsize": 10,
    "xtick.labelsize": 10,
    "ytick.labelsize": 10,
    "axes.spines.top": False,
    "axes.spines.right": False,
    "figure.dpi": 300,
    "savefig.dpi": 300
})



In [32]:
def plot_all_variables(time, station, landuse,
                       swdown, swdown_qc,
                       lwdown, lwdown_qc,
                       precip, precip_qc,
                       wind, wind_qc,
                       vpd, vpd_qc,
                       output_base):

    # Create landuse subfolder
    landuse_dir = os.path.join(output_base, landuse)
    os.makedirs(landuse_dir, exist_ok=True)

    fig, axs = plt.subplots(5, 1, figsize=(12, 10), sharex=True)

    def plot_subplot(ax, var, qc, title, ylabel):
        ax.plot(time, var, '.', color='lightgray', markersize=1, alpha=0.5, label='All')
        ax.plot(time[qc == 0], var[qc == 0], '.', color='#1f77b4', markersize=2, label='QC==0')
        ax.plot(time[qc == 1], var[qc == 1], '.', color='#ff7f0e', markersize=2, label='QC==1')
        ax.set_ylabel(ylabel)
        ax.set_title(title, loc='left')
        ax.grid(True, linestyle='--', alpha=0.3)
        ax.legend(frameon=False, loc='upper right')

    plot_subplot(axs[0], swdown, swdown_qc, 'SWdown', 'SWdown [W/m²]')
    plot_subplot(axs[1], lwdown, lwdown_qc, 'LWdown', 'LWdown [W/m²]')
    plot_subplot(axs[2], precip, precip_qc, 'Precipitation', 'Precipitation [mm]')
    plot_subplot(axs[3], wind, wind_qc, 'Wind Speed', 'Wind Speed [m/s]')
    plot_subplot(axs[4], vpd, vpd_qc, 'VPD', 'VPD [hPa]')
    axs[4].set_xlabel('Time')

    fig.suptitle(f'{station} ({landuse}) — Forcing Variables with QC Flags', fontsize=14)
    fig.tight_layout(rect=[0, 0, 1, 0.96])

    filename = f"{station}_{landuse}_forcing_qc.png"
    plt.savefig(os.path.join(landuse_dir, filename))
    plt.close()

In [34]:

# Loop over all stations
for station in df['station_name'].unique():
    landuse = df[df['station_name'] == station]['IGBP_short_name'].values[0]
    print(f"{station} ({landuse}) processing...")

    f = [f for f in os.listdir(insitu_forcing_path) if f.endswith('.nc') and station in f]

    if f:
        ds = xr.open_dataset(os.path.join(insitu_forcing_path, f[0]))
        time = ds.time.values

        swdown = ds.SWdown.values.flatten()
        swdown_qc = ds.SWdown_qc.values.flatten()
        precip = ds.Precip.values.flatten()
        precip_qc = ds.Precip_qc.values.flatten()
        wind = ds.Wind.values.flatten()
        wind_qc = ds.Wind_qc.values.flatten()
        vpd = ds.VPD.values.flatten()
        vpd_qc = ds.VPD_qc.values.flatten()
        lwdown = ds.LWdown.values.flatten()
        lwdown_qc = ds.LWdown_qc.values.flatten()

        plot_all_variables(
            time, station, landuse,
            swdown, swdown_qc,
            lwdown, lwdown_qc,
            precip, precip_qc,
            wind, wind_qc,
            vpd, vpd_qc,
            output_path  # base path, subfolders are created inside
        )
    else:
        print(f"No dataset found for {station}.")


BE-Dor (GRA) processing...
IT-Tor (GRA) processing...
FR-Lqu (GRA) processing...
IT-Niv (GRA) processing...
FR-Tou (GRA) processing...
FR-Mej (GRA) processing...
FI-Sii (WET) processing...
CZ-wet (WET) processing...
SE-Deg (WET) processing...
IT-MBo (GRA) processing...
FR-CLt (GRA) processing...
FR-Lus (GRA) processing...
UK-Amo (WET) processing...
No dataset found for UK-Amo.
BE-Maa (CSH) processing...
FR-EM2 (CRO) processing...
DE-RuR (GRA) processing...
DE-Gri (GRA) processing...
DE-RuS (CRO) processing...
FR-Aur (CRO) processing...
BE-Lon (CRO) processing...
IT-BCi (CRO) processing...
SE-Sto (WET) processing...
DE-Amv (WET) processing...
IT-Noe (CSH) processing...
DE-Kli (CRO) processing...
FR-Gri (CRO) processing...
DE-GsB (GRA) processing...
DE-Geb (CRO) processing...
DE-Msr (ENF) processing...
IT-Lsn (OSH) processing...
DE-Hzd (DBF) processing...
DK-RCW (DBF) processing...
No dataset found for DK-RCW.
FR-Pue (EBF) processing...
FR-Bil (ENF) processing...
ES-LMa (SAV) processing.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/home/khanalp/code/PhD/preprocessICOSdata/csvs/01_stationinfo_L0_ICOS2025_with_heights.csv')

In [6]:
df['Site type'] = df['Site type'].str.title()


In [8]:
df.to_csv('/home/khanalp/code/PhD/preprocessICOSdata/csvs/01_stationinfo_L0_ICOS2025_with_heights.csv')

In [39]:
df_modelrun_paths = pd.read_csv('/home/khanalp/code/PhD/preprocessICOSdata/csvs/ICOS2025_modelrun_paths.csv')

In [40]:
df_modelrun_paths['config_path'][0]

'/home/khanalp/paper01/final/GRA/BE-Dor/input/BE-Dor_2025-07-25-2039/BE-Dor_2025-07-25-2039_config.txt'

In [41]:
import os

## Add folder and modify config files

In [42]:
import os
import pandas as pd

# Load model paths
df_model_paths = pd.read_csv("/home/khanalp/code/PhD/preprocessICOSdata/csvs/ICOS2025_modelrun_paths.csv")

variants = ['vbigleaf', 'vMLWV', 'vMLHF']

# Prepare empty columns in df_model_paths for new config paths
for v in variants:
    df_model_paths[f'config_path_{v}'] = None

for idx, row in df_model_paths.iterrows():
    original_config = row['config_path']
    
    # Read original config file
    try:
        with open(original_config, 'r') as f:
            lines = f.readlines()
    except FileNotFoundError:
        print(f"Config file not found: {original_config}, skipping.")
        continue

    last_line = lines[-1].strip()
    if not last_line.startswith("OutputPath="):
        print(f"Unexpected format in config file: {original_config}, skipping.")
        continue
    original_output_path = last_line.split("=", 1)[1]

    base_config_path = os.path.splitext(original_config)[0]

    for variant in variants:
        new_output_path = f"{os.path.join(original_output_path, variant)}/"
        os.makedirs(new_output_path, exist_ok=True)

        modified_lines = lines[:-1] + [f"OutputPath={new_output_path}\n"]

        new_config_path = f"{base_config_path}_{variant}.txt"

        with open(new_config_path, 'w') as f:
            f.writelines(modified_lines)

        print(f"Created: {new_config_path}")
        print(f"Output folder ensured: {new_output_path}")

        # Save new config path in the DataFrame
        df_model_paths.at[idx, f'config_path_{variant}'] = new_config_path

# Optionally save updated DataFrame back to CSV
output_csv = "/home/khanalp/code/PhD/preprocessICOSdata/csvs/ICOS2025_modelrun_paths_with_variants.csv"
df_model_paths.to_csv(output_csv, index=False)
print(f"Updated model paths with variants saved to: {output_csv}")


Created: /home/khanalp/paper01/final/GRA/BE-Dor/input/BE-Dor_2025-07-25-2039/BE-Dor_2025-07-25-2039_config_vbigleaf.txt
Output folder ensured: /home/khanalp/paper01/final/GRA/BE-Dor/output/BE-Dor_2025-07-25-2039/vbigleaf/
Created: /home/khanalp/paper01/final/GRA/BE-Dor/input/BE-Dor_2025-07-25-2039/BE-Dor_2025-07-25-2039_config_vMLWV.txt
Output folder ensured: /home/khanalp/paper01/final/GRA/BE-Dor/output/BE-Dor_2025-07-25-2039/vMLWV/
Created: /home/khanalp/paper01/final/GRA/BE-Dor/input/BE-Dor_2025-07-25-2039/BE-Dor_2025-07-25-2039_config_vMLHF.txt
Output folder ensured: /home/khanalp/paper01/final/GRA/BE-Dor/output/BE-Dor_2025-07-25-2039/vMLHF/
Created: /home/khanalp/paper01/final/GRA/IT-Tor/input/IT-Tor_2025-07-25-2039/IT-Tor_2025-07-25-2039_config_vbigleaf.txt
Output folder ensured: /home/khanalp/paper01/final/GRA/IT-Tor/output/IT-Tor_2025-07-25-2039/vbigleaf/
Created: /home/khanalp/paper01/final/GRA/IT-Tor/input/IT-Tor_2025-07-25-2039/IT-Tor_2025-07-25-2039_config_vMLWV.txt
Output 

In [43]:
# Merge df_model_paths with df to add IGBP_short_name based on station_name
df_model_paths = df_model_paths.merge(
    df[['station_name', 'IGBP_short_name']],
    on='station_name',
    how='left'  # Use 'left' to keep all rows from df_model_paths even if no match in df
)

# Now df_model_paths has the IGBP_short_name column added


## Create text file for running model parallely

In [44]:


variants = ['vMLWV', 'vMLHF', 'vbigleaf']

# Get unique IGBP_short_name values (dropna if you want)
igbp_types = df_model_paths['IGBP_short_name'].dropna().unique()

output_dir = '/home/khanalp/paper01/'  # folder to save txt files
os.makedirs(output_dir, exist_ok=True)

for igbp in igbp_types:
    for variant in variants:
        col_name = f'config_path_{variant}'
        
        # Select paths for this IGBP and variant, drop missing values
        paths = df_model_paths.loc[
            df_model_paths['IGBP_short_name'] == igbp, col_name
        ].dropna().tolist()
        
        # Create output filename, e.g. ENF_config_path_vMLWV.txt
        filename = f"{igbp}_config_path_{variant}.txt"
        filepath = os.path.join(output_dir, filename)
        
        # Write to file
        with open(filepath, 'w') as f:
            for path in paths:
                f.write(path + "\n")
        
        print(f"Saved {len(paths)} paths for {igbp} - {variant} to {filepath}")


Saved 12 paths for GRA - vMLWV to /home/khanalp/paper01/GRA_config_path_vMLWV.txt
Saved 12 paths for GRA - vMLHF to /home/khanalp/paper01/GRA_config_path_vMLHF.txt
Saved 12 paths for GRA - vbigleaf to /home/khanalp/paper01/GRA_config_path_vbigleaf.txt
Saved 5 paths for WET - vMLWV to /home/khanalp/paper01/WET_config_path_vMLWV.txt
Saved 5 paths for WET - vMLHF to /home/khanalp/paper01/WET_config_path_vMLHF.txt
Saved 5 paths for WET - vbigleaf to /home/khanalp/paper01/WET_config_path_vbigleaf.txt
Saved 1 paths for CSH - vMLWV to /home/khanalp/paper01/CSH_config_path_vMLWV.txt
Saved 1 paths for CSH - vMLHF to /home/khanalp/paper01/CSH_config_path_vMLHF.txt
Saved 1 paths for CSH - vbigleaf to /home/khanalp/paper01/CSH_config_path_vbigleaf.txt
Saved 7 paths for CRO - vMLWV to /home/khanalp/paper01/CRO_config_path_vMLWV.txt
Saved 7 paths for CRO - vMLHF to /home/khanalp/paper01/CRO_config_path_vMLHF.txt
Saved 7 paths for CRO - vbigleaf to /home/khanalp/paper01/CRO_config_path_vbigleaf.txt
S